In [1]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential  # Corrected import
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
import tkinter as tk
from tkinter import filedialog
from PIL import Image, ImageTk  # Corrected capitalization of ImageTk
import numpy as np


In [2]:
train_dir = r'C:\Users\hp\Downloads\archive (1)\train'
test_dir = r'C:\Users\hp\Downloads\archive (1)\test'
valid_dir = r'C:\Users\hp\Downloads\archive (1)\valid'

In [3]:
#set up for loading images
train_datagen = ImageDataGenerator(rescale=1./255)
test_datagen = ImageDataGenerator(rescale=1./255)
valid_datagen = ImageDataGenerator(rescale=1./255)

#load images from directory
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(64, 64),
    batch_size=32,
    class_mode='binary'
)
test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=(64, 64),
    batch_size=32,
    class_mode='binary'
)
valid_generator = valid_datagen.flow_from_directory(
    valid_dir,
    target_size=(64, 64),
    batch_size=32,
    class_mode='binary'
)


Found 30250 images belonging to 2 classes.
Found 6300 images belonging to 2 classes.
Found 6300 images belonging to 2 classes.


In [4]:
#building a simple CNN model 
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(64, 64, 3)), #32 no of nnuerons,(3,3) matrix size, relu as same
    MaxPooling2D(pool_size=(2, 2)),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D(pool_size=(2, 2)),
    Flatten(), #multi dimention convert to 1 d array
    Dense(128, activation='relu'),
    Dropout(0.5), #level of dropout..0.2 to 0.5
    Dense(1, activation='sigmoid') #binary classification: wildfire or no wildfire
])

#compile the model 
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy']) #classification problem so acuracy

C:\Users\hp\anaconda3\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [5]:
#if images are corrupted...we can skip them using
from PIL import ImageFile
ImageFile.LOAD_TRUNCATED_IMAGES = True

### Training Model

In [7]:
history = model.fit(train_generator,validation_data=valid_generator,epochs=2,verbose=1)

Epoch 1/2


C:\Users\hp\anaconda3\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


946/946 ━━━━━━━━━━━━━━━━━━━━ 57s 59ms/step - accuracy: 0.8335 - loss: 0.3506 - val_accuracy: 0.9411 - val_loss: 0.1535
Epoch 2/2
946/946 ━━━━━━━━━━━━━━━━━━━━ 54s 57ms/step - accuracy: 0.9308 - loss: 0.1914 - val_accuracy: 0.9478 - val_loss: 0.1404


In [8]:
model.save("ffd_model.keras")

In [10]:
from tensorflow.keras.models import load_model
model = load_model("ffd_model.keras")
print("Model Loaded")

Model Loaded


### Build Gui

In [13]:
from tensorflow.keras.models import load_model

In [ ]:
# function to 
# Function to load and predict an image
def predict_image():
    # Open file dialog to select an image
    file_path = filedialog.askopenfilename()
    if file_path:
        # Display the image in the GUI
        img = Image.open(file_path)
        img = img.resize((200, 200))
        img = ImageTk.PhotoImage(img)    #convert image for tk
        image_label.configure(image=img) #update the image in GUI
        image_label.image = img

        # Preprocess the image for the model
        img_for_model = Image.open(file_path).resize((64, 64))
        img_array = np.array(img_for_model) / 255.0  # Rescale like during training
        img_array = np.expand_dims(img_array, axis=0)  # Add batch dimension

        # Make a prediction
        prediction = model.predict(img_array)[0][0] #extracts the scalar prediction value
        result = "Wildfire" if prediction > 0.5 else "No Wildfire"
        result_label.config(text="Prediction: " + result)

# Setting up the GUI window
root = tk.Tk()
root.title("Forest Fire Detection")
root.geometry("400x400")

# Add widgets
btn = tk.Button(root, text="Upload Image", command=predict_image) #button triggers the predict_image() function when clicked
btn.pack(pady=20)

#Placeholder for displaying the selected image
image_label = tk.Label(root)
image_label.pack()

#Label to display the prediction result
result_label = tk.Label(root, text="Prediction: ", font=("Helvetica", 16))
result_label.pack(pady=20)

#Starts the Tkinter event loop, keeping the GUI active until manually closed
root.mainloop()

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 70ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
